In [1]:
import sys
from glob import glob

import zarr
import numpy as np
import pandas as pd

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from datetime import datetime, timedelta

In [5]:
import subprocess

In [6]:
data_pos_v3 = zarr.load(save_dir_scratch+'HRRR_clean_pos_v3.zarr')
data_neg_v3 = zarr.load(save_dir_scratch+'HRRR_clean_neg_v3.zarr')

data_pos_v4 = zarr.load(save_dir_scratch+'HRRR_clean_pos_v4.zarr')
data_neg_v4 = zarr.load(save_dir_scratch+'HRRR_clean_neg_v4.zarr')

In [7]:
shape_pos_v3 = data_pos_v3.shape
shape_neg_v3 = data_neg_v3.shape

In [8]:
batch_dir = '/glade/scratch/ksha/DATA/NCAR_batch/'

In [9]:
data_pos_v3.shape

(621683, 18, 21)

In [58]:
HRRRv3 = np.vstack([data_pos_v3, data_neg_v3])

In [59]:
MEANs = []
STDs = []

for i in range(20):
    MEANs.append(np.mean(HRRRv3[..., i]))
    STDs.append(np.std(HRRRv3[..., i]))
    
MEANs = np.array(MEANs)
STDs = np.array(STDs)

In [60]:
MEANs

array([ 3.32744267e+00,  2.30268275e+00,  3.49134700e-04,  3.61186403e+00,
       -2.15238651e-02,  1.03665838e+01,  4.41590603e-01,  1.79305680e-02,
        2.78253191e+03,  2.64969288e+02,  2.23630493e+02, -2.40273726e+01,
        5.40289715e+02,  1.86496654e+01,  7.37603012e+00,  1.10030807e+02,
        4.82801404e+01,  2.84954553e+02,  2.77329110e+02,  9.31139490e+04])

In [73]:
STDs

array([1.25938123e+01, 7.57117064e+00, 3.55973598e-04, 4.48313802e+00,
       3.13906268e-02, 4.16004674e+00, 1.48719440e+00, 1.13531478e-01,
       7.15634363e+03, 5.89775100e+02, 5.44153845e+02, 5.80314013e+01,
       1.05153825e+03, 1.00278741e+01, 4.01217916e+00, 9.86360084e+01,
       7.17775328e+01, 1.10908803e+01, 1.07798139e+01, 7.22807319e+03])

In [71]:
subprocess.Popen("rm {}TRAIN*.npy".format(batch_dir), shell=True)

<Popen: returncode: None args: 'rm /glade/scratch/ksha/DATA/NCAR_batch/TRAIN...>

In [72]:
cut = 30000

#subprocess.Popen("rm {}TRAIN*.npy".format(batch_dir), shell=True)

shape_pos_v3 = data_pos_v3[cut:, ...].shape
shape_neg_v3 = data_neg_v3[cut:, ...].shape

for i in range(shape_pos_v3[0]):
    slice_ = data_pos_v3[i, ...]
    
    for j in range(20):
        slice_[..., j] = (slice_[..., j] - MEANs[j]) / STDs[j]
        
    slice_[:, -1] = slice_[:, -1] > 0
    np.save(batch_dir+'TRAIN_pos_{}.npy'.format(i), slice_)
    
for i in range(shape_neg_v3[0]):
    slice_ = data_neg_v3[i, ...]
    
    for j in range(20):
        slice_[..., j] = (slice_[..., j] - MEANs[j]) / STDs[j]
        
    slice_[:, -1] = slice_[:, -1] > 0
    
    np.save(batch_dir+'TRAIN_neg_{}.npy'.format(i), slice_)

In [66]:
data = np.load(batch_dir+'TRAIN_pos_2.npy')

In [44]:
data.shape

(18, 21)